In [ ]:
#Data Data Cleaning and Exploratory Analysis

In [3]:
#Link vídeo da Análise original, professora Patrícia Siqueira Ramos - UFAL
#https://www.youtube.com/watch?v=0SgghZ5S87o
#Link dados Rural:
#URL: https://dados.agricultura.gov.br/dataset/sisser3\
#DICIONÁRIO DADOS: https://dados.agricultura.gov.br/dataset/baefdc68-9bad-4204-83e8-f2888b79ab48/resource/2c5c55d0-1473-4749-b08f-cfaf887a9fa3/download/dicionariodedados-sisser.pdf


NameError: name 'README' is not defined

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

pd.options.display.float_format = '{:.2f}'.format #para os números aparecerem com duas casas decimais
pd.set_option('display.max_columns', 50) #mostra 50 colunas ou o número desejado

In [5]:
dados = pd.read_excel("dados-2016-2021.xlsx", decimal=',')
dados.shape

(697453, 38)

In [6]:
#salvando os dados de 2021
dados21 = dados[dados['ANO_APOLICE'] == 2021]
dados21.shape

(212839, 38)

In [ ]:
#Cada uma das 212839 linhas, é uma apólice contratada por um produtor. 

In [7]:
dados21.head

<bound method NDFrame.head of                                 NM_RAZAO_SOCIAL  CD_PROCESSO_SUSEP  \
484614                         Newe Seguros S.A  15414900516201718   
484615           BRASILSEG COMPANHIA DE SEGUROS  15414001178200504   
484616               Mapfre Seguros Gerais S.A.  15414900586201342   
484617  FairFax Brasil Seguros Corporativos S/A  15414004195201052   
484618                         Too Seguros S.A.  15414900961201869   
...                                         ...                ...   
697448           BRASILSEG COMPANHIA DE SEGUROS  15414609027202020   
697449           BRASILSEG COMPANHIA DE SEGUROS  15414001178200504   
697450           BRASILSEG COMPANHIA DE SEGUROS  15414001178200504   
697451           BRASILSEG COMPANHIA DE SEGUROS  15414001178200504   
697452           BRASILSEG COMPANHIA DE SEGUROS  15414001178200504   

                 NR_PROPOSTA  ID_PROPOSTA DT_PROPOSTA DT_INICIO_VIGENCIA  \
484614                 38275      1289687  2020-12-03

In [8]:
#cópia de segurança, vamos trabalhar com uma cópia dos dados, não com o conjunto original
seg = dados21.copy()

In [9]:
seg.head()

,NM_RAZAO_SOCIAL,CD_PROCESSO_SUSEP,NR_PROPOSTA,ID_PROPOSTA,DT_PROPOSTA,DT_INICIO_VIGENCIA,DT_FIM_VIGENCIA,NM_SEGURADO,NR_DOCUMENTO_SEGURADO,NM_MUNICIPIO_PROPRIEDADE,SG_UF_PROPRIEDADE,LATITUDE,NR_GRAU_LAT,NR_MIN_LAT,NR_SEG_LAT,LONGITUDE,NR_GRAU_LONG,NR_MIN_LONG,NR_SEG_LONG,NR_DECIMAL_LATITUDE,NR_DECIMAL_LONGITUDE,NM_CLASSIF_PRODUTO,NM_CULTURA_GLOBAL,NR_AREA_TOTAL,NR_ANIMAL,NR_PRODUTIVIDADE_ESTIMADA,NR_PRODUTIVIDADE_SEGURADA,NivelDeCobertura,VL_LIMITE_GARANTIA,VL_PREMIO_LIQUIDO,PE_TAXA,VL_SUBVENCAO_FEDERAL,NR_APOLICE,DT_APOLICE,ANO_APOLICE,CD_GEOCMU,VALOR_INDENIZAÇÃO,EVENTO_PREPONDERANTE
484614,Newe Seguros S.A,15414900516201718,38275,1289687,2020-12-03,2021-02-15,2021-08-15 00:00:00,SUAIR FERREIRA DE MENDONCA,***51059168,Rio Verde,GO,S,18,5,10,W,50,30,17,-,-,CUSTEIO,Milho 2ª safra,62.00,0,6171.60,4011.60,0.65,133064.77,7895.27,0.06,3158.11,10001010026768,2021-01-11,2021,5218805,17583.39,SECA
484615,BRASILSEG COMPANHIA DE SEGUROS,15414001178200504,261484155,1289688,2020-11-20,2020-11-20,2021-11-20 00:00:00,LEANDRO RUSCITTI DINIZ,***11954119,Jataí,GO,S,17,55,27,W,52,9,55,-,-,CUSTEIO,Milho 2ª safra,268.00,0,6013,3908.45,0.65,608223.32,33452.28,0.05,13380.91,494404,2021-01-04,2021,5211909,-,-
484616,Mapfre Seguros Gerais S.A.,15414900586201342,66076867965456,1289689,2020-11-18,2020-11-18,2021-07-20 00:00:00,ALIPIO DIVINO BORGES PORTILHO,***37790159,São Félix do Araguaia,MT,S,11,23,12,W,52,13,59,-,-,PRODUTIVIDADE,Milho 2ª safra,850.00,0,6276,4079.40,0.65,2311660.00,195224.61,0.08,48000.00,4525000065701,2021-01-04,2021,5107859,-,-
484617,FairFax Brasil Seguros Corporativos S/A,15414004195201052,10000116410000327304,1289690,2020-12-02,2020-12-02,2021-09-06 00:00:00,ADRIANO CAETANO COSTA,***80051957,Doutor Camargo,PR,S,23,31,48,W,52,13,18,-,-,PRODUTIVIDADE,Milho 2ª safra,15.73,0,5569.80,3620.40,0.65,47457.41,5277.26,0.11,2110.90,1000100040113,2021-01-08,2021,4107306,16413.80,SECA
484618,Too Seguros S.A.,15414900961201869,23530,1289691,2021-01-06,2021-03-01,2021-08-28 00:00:00,EDSON EMILIO GOMES,***99130997,São Sebastião da Amoreira,PR,S,23,25,6,W,50,47,38,-,-,PRODUTIVIDADE,Milho 2ª safra,124.97,0,5226,3658.20,0.70,304776.84,43755.91,0.14,17502.36,1000111016260,2021-01-12,2021,4126009,248902.36,SECA


In [10]:
seg.tail()

,NM_RAZAO_SOCIAL,CD_PROCESSO_SUSEP,NR_PROPOSTA,ID_PROPOSTA,DT_PROPOSTA,DT_INICIO_VIGENCIA,DT_FIM_VIGENCIA,NM_SEGURADO,NR_DOCUMENTO_SEGURADO,NM_MUNICIPIO_PROPRIEDADE,SG_UF_PROPRIEDADE,LATITUDE,NR_GRAU_LAT,NR_MIN_LAT,NR_SEG_LAT,LONGITUDE,NR_GRAU_LONG,NR_MIN_LONG,NR_SEG_LONG,NR_DECIMAL_LATITUDE,NR_DECIMAL_LONGITUDE,NM_CLASSIF_PRODUTO,NM_CULTURA_GLOBAL,NR_AREA_TOTAL,NR_ANIMAL,NR_PRODUTIVIDADE_ESTIMADA,NR_PRODUTIVIDADE_SEGURADA,NivelDeCobertura,VL_LIMITE_GARANTIA,VL_PREMIO_LIQUIDO,PE_TAXA,VL_SUBVENCAO_FEDERAL,NR_APOLICE,DT_APOLICE,ANO_APOLICE,CD_GEOCMU,VALOR_INDENIZAÇÃO,EVENTO_PREPONDERANTE
697448,BRASILSEG COMPANHIA DE SEGUROS,15414609027202020,266305260,1524279,2021-09-09,2021-09-09,2022-09-09 00:00:00,ELMAR PAULO B MACHADO,***83115072,Formoso,MG,S,15,7,4,W,46,19,56,-,-,CUSTEIO,Milho 2ª safra,372.00,-,6000,3900,0.65,2624647.14,174539.03,0.07,17575.21,178989,2021-09-17,2021,3126208,-,-
697449,BRASILSEG COMPANHIA DE SEGUROS,15414001178200504,266643597,1524280,2021-09-28,2021-09-28,2022-09-28 00:00:00,MARCOS RODRIGUES,***93179187,Girassol,GO,S,15,28,16,W,48,41,24,-,-,CUSTEIO,Milho 2ª safra,341.00,-,6084,3954.60,0.65,1016132.26,65189.96,0.06,25127.74,181738,2021-09-30,2021,-,-,-
697450,BRASILSEG COMPANHIA DE SEGUROS,15414001178200504,266643663,1524281,2021-09-28,2021-09-28,2022-09-28 00:00:00,DIONE MOREIRA SILVA,***81735102,Niquelândia,GO,S,14,23,52,W,48,35,11,-,-,CUSTEIO,Milho 2ª safra,160.00,-,4010,2606.50,0.65,428411.36,27846.73,0.06,11138.68,182550,2021-09-30,2021,5214606,-,-
697451,BRASILSEG COMPANHIA DE SEGUROS,15414001178200504,266412506,1524282,2021-09-15,2021-09-15,2022-09-15 00:00:00,ONDI WEBER,***42020990,Itaporã,MS,S,21,51,51,W,54,49,35,-,-,CUSTEIO,Milho 2ª safra,278.00,-,6000,3900,0.65,1100603.97,104557.38,0.10,41822.95,179849,2021-09-21,2021,5004502,-,-
697452,BRASILSEG COMPANHIA DE SEGUROS,15414001178200504,266643757,1524283,2021-09-28,2021-09-28,2022-09-28 00:00:00,MARLI TEREZINHA DE SOUZA CARDOSO,***08373987,Itaipulândia,PR,S,25,12,7,W,54,13,24,-,-,CUSTEIO,Milho 2ª safra,162.00,-,5922,3849.30,0.65,610999.84,65758.86,0.11,25382.18,182278,2021-09-30,2021,4110953,-,-


In [12]:
seg.sample(10)

,NM_RAZAO_SOCIAL,CD_PROCESSO_SUSEP,NR_PROPOSTA,ID_PROPOSTA,DT_PROPOSTA,DT_INICIO_VIGENCIA,DT_FIM_VIGENCIA,NM_SEGURADO,NR_DOCUMENTO_SEGURADO,NM_MUNICIPIO_PROPRIEDADE,SG_UF_PROPRIEDADE,LATITUDE,NR_GRAU_LAT,NR_MIN_LAT,NR_SEG_LAT,LONGITUDE,NR_GRAU_LONG,NR_MIN_LONG,NR_SEG_LONG,NR_DECIMAL_LATITUDE,NR_DECIMAL_LONGITUDE,NM_CLASSIF_PRODUTO,NM_CULTURA_GLOBAL,NR_AREA_TOTAL,NR_ANIMAL,NR_PRODUTIVIDADE_ESTIMADA,NR_PRODUTIVIDADE_SEGURADA,NivelDeCobertura,VL_LIMITE_GARANTIA,VL_PREMIO_LIQUIDO,PE_TAXA,VL_SUBVENCAO_FEDERAL,NR_APOLICE,DT_APOLICE,ANO_APOLICE,CD_GEOCMU,VALOR_INDENIZAÇÃO,EVENTO_PREPONDERANTE
645039,BRASILSEG COMPANHIA DE SEGUROS,15414609027202020,264744624,1469759,2021-06-17,2021-06-17,2022-06-17 00:00:00,JOVINO TOTTI,***78096887,Assis,SP,S,22,40,29,W,50,21,53,-,-,CUSTEIO,Soja,78.00,-,2800,1820,0.65,198673.80,14217.05,0.07,2843.41,150831,2021-06-30,2021,3504008,-,-
693219,Sancor Seguros do Brasil S.A.,15414901493201769,20214447,1519962,2021-12-22,2021-12-22,2022-07-19 00:00:00,VITOR MATEUS DUFFECK,***72749902,Corbélia,PR,S,24,55,21,W,53,13,3,-,-,PRODUTIVIDADE,Milho 2ª safra,30.44,0,6087.40,4261.18,0.70,151320.06,20175.03,0.13,8070.01,1000111096702,2021-12-23,2021,4106308,-,-
513347,Essor Seguros S.A.,15414004513201247,118111714874065,1322598,2021-03-10,2021-03-10,2021-10-15 00:00:00,JOAO LUCAS MARCUSSO PIRES,***25426981,Ribeirão do Sul,SP,S,22,46,33,W,49,59,59,-,-,PRODUTIVIDADE,Milho 2ª safra,122.88,0,5723.08,3720,0.65,457113.60,54853.63,0.12,21941.45,1000111141928,2021-03-29,2021,3543204,284324.66,GEADA
672784,Aliança do Brasil Seguros S/A.,15414901479201927,55075584,1499154,2021-09-21,2021-09-21,2022-09-21 00:00:00,ELKE MARINA LEH BASSO,***82618904,Pinhão,PR,S,25,47,25,W,51,47,17,-,-,CUSTEIO,Milho 1ª safra,35.52,0,12000,8400,0.70,230880.01,7721.65,0.03,1930.41,22527,2021-09-27,2021,4119301,51332.79,SECA
694482,BRASILSEG COMPANHIA DE SEGUROS,15414609027202020,267034494,1521296,2021-10-20,2021-10-20,2022-10-20 00:00:00,MARIA BERNADETHE LUNA GRANJA,***33520161,Itaporã,MS,S,21,53,14,W,54,59,17,-,-,CUSTEIO,Milho 2ª safra,70.00,-,6600,4290,0.65,277675.58,27767.56,0.10,11107.02,186828,2021-10-25,2021,5004502,-,-
626918,FairFax Brasil Seguros Corporativos S/A,15414004195201052,10000116410000354099,1449916,2021-08-26,2021-08-26,2022-04-14 00:00:00,GUSTAVO XAVIER DOS ANJOS,***65705919,Jussara,PR,S,23,37,27,W,52,25,37,-,-,PRODUTIVIDADE,Soja,44.04,0,3480,2436,0.70,286083.84,19367.88,0.07,3873.58,1000100052346,2021-09-20,2021,4113007,227035.01,SECA
487558,Newe Seguros S.A,15414900516201718,39289,1293045,2021-01-18,2021-02-20,2021-08-20 00:00:00,ANIELLY MATIUCI BERNARDES,***69961971,Goioerê,PR,S,24,9,59,W,53,0,45,-,-,CUSTEIO,Milho 2ª safra,34.30,0,6000,3900,0.65,156065.00,17947.48,0.12,7178.99,10001010028716,2021-03-09,2021,4108601,50144.89,SECA
659219,Mapfre Seguros Gerais S.A.,15414900521201305,44074491162706,1485042,2021-10-11,2021-10-11,2022-10-11 00:00:00,GERONCIO BRAGA DE SOUZA,***51183607,Durandé,MG,S,20,5,14,W,41,49,15,-,-,CUSTEIO,Café,2.75,0,0,0,-,33000.00,330.00,0.01,132.00,6609002484701,2021-10-14,2021,3123528,-,-
555401,Essor Seguros S.A.,15414004513201247,118111925048052,1369183,2021-05-12,2021-06-01,2022-04-30 00:00:00,AMADEU BORGES DA SILVA,***99424053,Monte Alegre dos Campos,RS,S,28,47,14,W,50,50,1,-,-,PRODUTIVIDADE,Uva,1.32,0,30500,30500,1,50727.60,3550.94,0.07,1420.38,1000111148965,2021-05-21,2021,4312377,-,-
696055,BRASILSEG COMPANHIA DE SEGUROS,15414609027202020,266388838,1522873,2021-09-14,2021-09-14,2022-09-14 00:00:00,MARCOS ANTONIO BERTOLI JUNIOR,***90383995,Ibaté,SP,S,22,0,0,W,48,0,0,-,-,CUSTEIO,Milho 2ª safra,20.57,-,6000,3900,0.65,101089.55,10614.40,0.10,4245.76,178753,2021-09-16,2021,3519303,-,-


In [17]:
seg.columns #mostra as variáveis que temos no conjunto de dados

Index(['NM_RAZAO_SOCIAL', 'CD_PROCESSO_SUSEP', 'NR_PROPOSTA', 'ID_PROPOSTA',
       'DT_PROPOSTA', 'DT_INICIO_VIGENCIA', 'DT_FIM_VIGENCIA', 'NM_SEGURADO',
       'NR_DOCUMENTO_SEGURADO', 'NM_MUNICIPIO_PROPRIEDADE',
       'SG_UF_PROPRIEDADE', 'LATITUDE', 'NR_GRAU_LAT', 'NR_MIN_LAT',
       'NR_SEG_LAT', 'LONGITUDE', 'NR_GRAU_LONG', 'NR_MIN_LONG', 'NR_SEG_LONG',
       'NR_DECIMAL_LATITUDE', 'NR_DECIMAL_LONGITUDE', 'NM_CLASSIF_PRODUTO',
       'NM_CULTURA_GLOBAL', 'NR_AREA_TOTAL', 'NR_ANIMAL',
       'NR_PRODUTIVIDADE_ESTIMADA', 'NR_PRODUTIVIDADE_SEGURADA',
       'NivelDeCobertura', 'VL_LIMITE_GARANTIA', 'VL_PREMIO_LIQUIDO',
       'PE_TAXA', 'VL_SUBVENCAO_FEDERAL', 'NR_APOLICE', 'DT_APOLICE',
       'ANO_APOLICE', 'CD_GEOCMU', 'VALOR_INDENIZAÇÃO',
       'EVENTO_PREPONDERANTE'],
      dtype='object')

In [21]:
seg.info() #algumas variáveis, como DT_FIM_VIGENCIA vão precisar ter o seu dtype alterado. 

<class 'pandas.core.frame.DataFrame'>
Index: 212839 entries, 484614 to 697452
Data columns (total 38 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   NM_RAZAO_SOCIAL            212839 non-null  object        
 1   CD_PROCESSO_SUSEP          212839 non-null  int64         
 2   NR_PROPOSTA                212839 non-null  object        
 3   ID_PROPOSTA                212839 non-null  int64         
 4   DT_PROPOSTA                212839 non-null  datetime64[ns]
 5   DT_INICIO_VIGENCIA         212839 non-null  datetime64[ns]
 6   DT_FIM_VIGENCIA            212839 non-null  object        
 7   NM_SEGURADO                212839 non-null  object        
 8   NR_DOCUMENTO_SEGURADO      212839 non-null  object        
 9   NM_MUNICIPIO_PROPRIEDADE   212839 non-null  object        
 10  SG_UF_PROPRIEDADE          212839 non-null  object        
 11  LATITUDE                   212839 non-null  object  

NoneType

In [ ]:
FILTRANDO LINHAS #fazendo apenas consultas na base